In [1]:
import os
import sys
import pandas as pd
import requests
from pandasai import SmartDataframe, SmartDatalake
from pandasai.llm.local_llm import LocalLLM 

In [2]:
API_BASE = "https://api.sambanova.ai/v1"
MODEL = "Meta-Llama-3.1-70B-Instruct"
API_KEY = "540f8914-997e-46c6-829a-ff76f5d4d265"

In [3]:
def load_file(file_path):
    try:
        if file_path.endswith('.csv'):
            return pd.read_csv(file_path, encoding='utf-8')
        elif file_path.endswith(('.xlsx', '.xls')):
            return pd.read_excel(file_path)
        else:
            print("Unsupported file format. Please use CSV or Excel files.")
            return None
    except Exception as e:
        print(f"Error loading file: {str(e)}")
        return None

In [4]:
def load_multiple_files():
    dataframes = []
    while True:
        file_path = input("Enter File Path (or 'done' when finished): ").strip()
        if file_path.lower() == 'done':
            break
        
        df = load_file(file_path)
        if df is not None:
            dataframes.append(df)
            print(f"Successfully loaded: {file_path}")
    
    return dataframes

In [9]:
def detect_data_issues(df):
    """
    Analyze the DataFrame for common data issues and return a description of found issues.
    """
    issues = []
    
    missing_values = df.isnull().sum()
    if missing_values.any():
        missing_cols = missing_values[missing_values > 0]
        issues.append(f"Missing values found in columns: {', '.join(missing_cols.index)} "
                     f"(counts: {', '.join(map(str, missing_cols.values))})")
    
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        issues.append(f"Found {duplicates} duplicate rows")
    
    mixed_type_cols = []
    for col in df.columns:
        if df[col].dtype == 'object':
            numeric_count = pd.to_numeric(df[col], errors='coerce').notna().sum()
            if 0 < numeric_count < len(df):
                mixed_type_cols.append(col)
    if mixed_type_cols:
        issues.append(f"Mixed data types found in columns: {', '.join(mixed_type_cols)}")
    
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)].shape[0]
        if outliers > 0:
            issues.append(f"Found {outliers} potential outliers in column '{col}'")
    
    string_cols = df.select_dtypes(include=['object']).columns
    for col in string_cols:
        if df[col].str.isupper().any() and df[col].str.islower().any():
            issues.append(f"Inconsistent string casing found in column '{col}'")
        if (df[col].str.len() != df[col].str.strip().str.len()).any():
            issues.append(f"Issues: '{col}'")
    
    return "Data Issues Found:\n" + "\n".join(f"- {issue}" for issue in issues) if issues else "No significant data issues found."


In [10]:
def process_with_llama(original_data, data_issues):
    """
    Process data with LLaMA 70B model and return modified data based on identified issues
    """
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    prompt = f"""Based on the following data and identified issues, provide a MODIFIED version of the original data that addresses all the issues. Return ONLY the modified data in a table format.

Original Data:
{original_data.to_string()}

Identified Data Issues:
{data_issues}

Dataset Info:
{original_data.info()}

IMPORTANT INSTRUCTIONS:
1. Address ALL the identified issues in your response
2. Return the complete modified dataset as a table
3. Include ALL relevant columns from the original data
4. Make necessary modifications to fix the identified issues
5. Use the same column names as the original data
6. Maintain proper data types for each column
7. Use '|' as column separator
8. Include a header row

Return ONLY the modified table with no additional text."""

    payload = {
        "messages": [
            {
                "role": "system",
                "content": "You are a data transformation assistant. Return only the modified data table with the same structure as the input data."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "model": MODEL,
        "max_tokens": 4000,
        "temperature": 0.3
    }

    try:
        response = requests.post(f"{API_BASE}/chat/completions", headers=headers, json=payload)
        if response.status_code != 200:
            print(f"API Error: {response.status_code}")
            print(f"Response: {response.text}")
            return None
        
        result = response.json()['choices'][0]['message']['content']
        table_lines = [line.strip() for line in result.strip().split('\n') if '|' in line]
        if not table_lines:
            return None
            
        # Convert the table string back to DataFrame
        try:
            # Split the header and clean it
            headers = [col.strip() for col in table_lines[0].split('|') if col.strip()]
            
            # Process data rows
            data = []
            for line in table_lines[2:]:  # Skip header and separator line
                row = [cell.strip() for cell in line.split('|') if cell.strip()]
                if row:  # Only add non-empty rows
                    data.append(row)
            
            # Create DataFrame with the same structure as original
            modified_df = pd.DataFrame(data, columns=headers)
            
            # Convert data types to match original DataFrame
            for col in modified_df.columns:
                if col in original_data.columns:
                    try:
                        modified_df[col] = modified_df[col].astype(original_data[col].dtype)
                    except:
                        pass  # Keep original type if conversion fails
            
            return modified_df
            
        except Exception as e:
            print(f"Error converting table to DataFrame: {str(e)}")
            return None
            
    except Exception as e:
        print(f"Error in LLaMA processing: {str(e)}")
        return None

In [18]:
def main():
    print("Please enter file paths one at a time. Type 'done' when finished.")
    dataframes = load_multiple_files()
    
    if not dataframes:
        print("No valid files were loaded. Exiting.")
        return

    combined_df = pd.concat(dataframes, axis=0, ignore_index=True)
    
    # Detect data issues
    print("\nAnalyzing data issues...")
    data_issues = detect_data_issues(combined_df)
    print("\nData Analysis Results:")
    print(data_issues)
    
    # Process with LLaMA to get modified data
    print("\nProcessing data with LLaMA to address identified issues...")
    modified_df = process_with_llama(combined_df, data_issues)
    
    if modified_df is not None:
        print("\nModified Data from LLaMA:")
        print(modified_df.to_string())
        print("\nData has been processed and issues have been addressed.")
    else:
        print("\nCould not get modified data from LLaMA.")

    custom_prompt = """You are a helpful assistant capable of processing and returning data in tabular form. Whenever a user asks for information, process the query and return the result as a well-structured table. Please ensure that the table includes the relevant columns, data, and is easy to read.

            Structure: Format the result as a table with clear headers.
            Data Representation: Ensure the data is returned in a structured, tabular form, with appropriate rows and columns.
            Context Understanding: If the query is about filtering, sorting, or aggregating data, return the modified table as per the user's request.
            Handle Missing Data: If applicable, indicate missing data as NaN or similar.
            Respond with the Table Only: Focus on returning the table, without unnecessary explanation.
    Query: {question}
    Please provide a clear and relevant response."""
    
    llm = LocalLLM(api_base=API_BASE, model=MODEL, api_key=API_KEY)
    smart_df = SmartDataframe(
        combined_df, 
        config={
            "llm": llm,
            "prompt_template": custom_prompt
        }
    )

    while True:
        query = input("Enter Query (or 'exit' to quit): ").strip()
        if query.lower() == 'exit':
            break
        else:
            try:
                query_result = smart_df.chat(query)
                print("\nInitial Query Result:", query_result)
                
                modified_df = process_with_llama(combined_df, data_issues)
                
                if modified_df is not None:
                    print("\nModified Data from LLaMA:")
                    print(modified_df.to_string())
                    # Update the combined_df with modified data
                    combined_df = modified_df
                    print("\nData has been updated with the modifications.")
                else:
                    print("\nCould not get modified data from LLaMA.")
                    
            except Exception as e:
                error_msg = str(e).encode('utf-8', errors='replace').decode('utf-8')
                print(f"Error: {error_msg}")


In [19]:
main()

Please enter file paths one at a time. Type 'done' when finished.
Successfully loaded: SuperApp-sampledata(in).csv

Analyzing data issues...

Data Analysis Results:
Data Issues Found:
- Missing values found in columns: No.of.Users, Software Owner, IT Owner, Existing Sub Domain, Regional (counts: 1, 2, 15, 6, 3)
- Mixed data types found in columns: No.of.Users
- Issues: 'No.of.Users'
- Issues: 'Software Owner'
- Issues: 'IT Owner'
- Issues: 'Existing Sub Domain'
- Inconsistent string casing found in column 'Locations'
- Issues: 'Regional'

Processing data with LLaMA to address identified issues...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Application                  66 non-null     object
 1   Supports                     66 non-null     object
 2   No.of.Users                  65 non-null     object
 3   Sof

IOException: IO Error: File is already open in 
C:\Users\moham\AppData\Local\Programs\Python\Python311\python.exe (PID 23860)